## The `app`
* interface to the Dash _application layer_
* manages application concerns, such as _rendering_ and _serving_
* has hooks for business concerns, such as _layout_ and _callbacks_
* Tip: If you are running JupyterLab in Anaconda, run the app inline: `mode='inline'`.

In [ ]:
from jupyter_dash import JupyterDash
import dash_html_components as html

external_stylesheets = [
    'https://codepen.io/chriddyp/pen/bWLwgP.css',
    'file://localhost/home/jovyan/work/assets/customer.css'
                       ]

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),
    html.Div(children='''
        Dash: A web application framework for Python.
    '''),
])

app.run_server(host='0.0.0.0', port='8050', mode='jupyterlab') # or mode='inline'

In [ ]:
def init_app():
    return JupyterDash(__name__, external_stylesheets=external_stylesheets)

def run_server(app):
    app.run_server(
        host='0.0.0.0', 
        port='8050', 
        mode='jupyterlab', 
        debug=False)

## Plotly

In [ ]:
import pandas as pd

# Valid URL schemes include http, ftp, s3, gs, and file.
df = pd.read_csv('file://localhost/home/jovyan/work/penguins.csv') # In Anaconda, change to local path to file.
df

In [ ]:
import plotly.express as px

fig = px.scatter(
    df, 
    x="bill_length_mm", 
    y="bill_depth_mm", 
    color="species"
)
fig.update_layout(title='Bill Shape')
fig.show()

## Dash Component - Graph

In [ ]:
import dash_core_components as dcc

app.layout = html.Div(children=[
    html.H1(children='Penguins'),
    dcc.Graph(
        id='scatter',
        figure=fig
    )
])

run_server(app)

## Dash Component - Input
* Supported types: text, number, password, email, search, tel, url, range, hidden

In [ ]:
app.layout = html.Div(children=[
    html.H1(children='Penguins'),
    html.Div(children=[
        html.Label(children="Title"),
        dcc.Input(
            id="input_title",
            type='text',
            value='Bill Shape'
        )
    ], id="container_form", className="pretty_container"),
    html.Div(children=[
        dcc.Graph(
            id='scatter',
            figure=fig
        )
    ], id="container_fig", className="pretty_container")
])

run_server(app)

## Callbacks

In [ ]:
import dash

app = init_app()

app.layout = html.Div(children=[
    html.H1(children='Penguins'),
    html.Div(children=[
        html.Label(children="Title"),
        dcc.Input(
            id="input_title",
            type='text',
            value='Bill Shape'
        )
    ], id="container_form", className="pretty_container"),
    html.Div([
        html.Div(id='output_container')
    ], id="container_fig", className="pretty_container")
])

@app.callback(
    dash.dependencies.Output('output_container', 'children'),
    [dash.dependencies.Input('input_title', 'value')]
)
def update_content(title):
    return f'Title: "{title}"'

run_server(app)

## Dash Component - Dropdown

In [ ]:
app = init_app()

app.layout = html.Div(children=[
    html.H1(children='Penguins'),
    html.Div(children=[
        html.Label(children="Title"),
        dcc.Input(
            id="input_title",
            type='text',
            value='Bill Shape'
        ),
        html.Label(children="Grouping"),
        dcc.Dropdown(
            id='dropdown_grouping',
            options=[
                {'label': 'Species', 'value': 'species'},
                {'label': 'Island', 'value': 'island'},
                {'label': 'Year', 'value': 'year'},
            ], 
            value='species'
        ),
        html.Label(children="X Dimension"),
        dcc.Dropdown(
            id='dropdown_x',
            options=[
                {'label': 'Bill Length (mm)', 'value': 'bill_length_mm'},
                {'label': 'Bill Depth (mm)', 'value': 'bill_depth_mm'},
                {'label': 'Flipper Length (mm)', 'value': 'flipper_length_mm'},
                {'label': 'Body Mass (g)', 'value': 'body_mass_g'},
            ], 
            value='bill_length_mm'
        ),
        html.Label(children="Y Dimension"),
        dcc.Dropdown(
            id='dropdown_y',
            options=[
                {'label': 'Bill Length (mm)', 'value': 'bill_length_mm'},
                {'label': 'Bill Depth (mm)', 'value': 'bill_depth_mm'},
                {'label': 'Flipper Length (mm)', 'value': 'flipper_length_mm'},
                {'label': 'Body Mass (g)', 'value': 'body_mass_g'},
            ], 
            value='bill_depth_mm'
        )
    ], id="container_form", className="pretty_container"),
    html.Div([
        html.Div(id='graph_container')
    ], id="container_fig", className="pretty_container")
])

run_server(app)

## Dynamic Graph

In [ ]:
@app.callback(
    dash.dependencies.Output('graph_container', 'children'),
    [
        dash.dependencies.Input('input_title', 'value'),
        dash.dependencies.Input('dropdown_grouping', 'value'),
        dash.dependencies.Input('dropdown_x', 'value'),
        dash.dependencies.Input('dropdown_y', 'value'),
    ]
)
def update_graph(title, grouping, x, y):
    fig = px.scatter(
        df, 
        x=x, 
        y=y, 
        color=grouping
    )
    fig.update_layout(title=title)
    return dcc.Graph(id='scatter', figure=fig)

run_server(app)